In [3]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Library Imports

import os
import gc
import ast
import json
import time
import uuid
import math
import traceback
import warnings
import urllib.parse
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
import psycopg2.extras as extras
warnings.filterwarnings("ignore")

In [2]:
#Database Parameters
database='isgpostgres'
user='postgres'
password='123@NjMpTs'
host='20.62.75.39'
port='9030'

connection=psycopg2.connect(database=database,user=user,password=password,host=host ,port=port)

#Global Variable
volume_table='tm_new_data.nys_hour_8_volume_240524'
connected_ids_table='tm_new_data.ny_connected_osm_master_231211'
output_table='nys_augmentation_hour_8_volume_231211'
schema='tm_new_data'

iteration=2
source='RMBA'

Missing_IDS=[]
error_dict={}

if source=='RMFA' : 
    tag='prev'
    tag1='ncc'
else:
    tag='next'
    tag1='pcc'


augmented_df=pd.DataFrame({'isg_osm_id':[],'source':[],'highway_id':[],'volume':[],'original_ids':[],'single_filled':[],'iteration':[]},dtype='int64')
selected_lst=['isg_osm_id','source','highway_id','volume','original_ids','single_filled','iteration']

#Read base data
sql_command=f"""
                select distinct isg_osm_id,highway_id,sum({tag}_volume) as volume,array_agg({tag}_isg_osm_id) connected_ids
                from
                    (select distinct isg_osm_id,{tag}_volume ,{tag}_isg_osm_id,highway_id,{tag}_connected_count,{tag}_{tag1}
                    from {volume_table} where volume = -1 and {tag}_connected_count > 1
                    )foo 
                where {tag}_volume != -1 group by isg_osm_id,highway_id  having count(distinct {tag}_isg_osm_id) = sum(distinct {tag}_connected_count) and sum(distinct {tag}_{tag1}) = 1
            """

all_osm_data_df=pd.read_sql(sql_command,connection)
ALL_Available_LST = np.unique(list(set(all_osm_data_df.to_dict(orient='list')['isg_osm_id']))).tolist()


print("Total Augmented Data Length :",len(all_osm_data_df))




Total Augmented Data Length : 88


In [3]:
start=time.time()
counter=0
for search_osm_id in ALL_Available_LST:
    try:
        extract_df=all_osm_data_df.loc[all_osm_data_df['isg_osm_id']==search_osm_id].reset_index(drop=True)
        extract_df['source']=source
        extract_df['original_ids']=search_osm_id
        extract_df['single_filled'] = None
        extract_df['iteration'] = iteration
        extract_df=extract_df[selected_lst]
    
        augmented_df=pd.concat([augmented_df,extract_df],ignore_index=True)
    
        counter=counter+1
        print("Processed ID:",counter)
            
    except:
        #print(traceback.format_exc())
        error_dict[str(search_osm_id)] = str(traceback.format_exc())
        Missing_IDS.append(search_osm_id)
        print("Some ID Missed")
        continue
    
end=time.time()

Processed ID: 1
Processed ID: 2
Processed ID: 3
Processed ID: 4
Processed ID: 5
Processed ID: 6
Processed ID: 7
Processed ID: 8
Processed ID: 9
Processed ID: 10
Processed ID: 11
Processed ID: 12
Processed ID: 13
Processed ID: 14
Processed ID: 15
Processed ID: 16
Processed ID: 17
Processed ID: 18
Processed ID: 19
Processed ID: 20
Processed ID: 21
Processed ID: 22
Processed ID: 23
Processed ID: 24
Processed ID: 25
Processed ID: 26
Processed ID: 27
Processed ID: 28
Processed ID: 29
Processed ID: 30
Processed ID: 31
Processed ID: 32
Processed ID: 33
Processed ID: 34
Processed ID: 35
Processed ID: 36
Processed ID: 37
Processed ID: 38
Processed ID: 39
Processed ID: 40
Processed ID: 41
Processed ID: 42
Processed ID: 43
Processed ID: 44
Processed ID: 45
Processed ID: 46
Processed ID: 47
Processed ID: 48
Processed ID: 49
Processed ID: 50
Processed ID: 51
Processed ID: 52
Processed ID: 53
Processed ID: 54
Processed ID: 55
Processed ID: 56
Processed ID: 57
Processed ID: 58
Processed ID: 59
Proces

In [4]:
print("Time Taken:",(end-start))

Time Taken: 0.15452933311462402


In [5]:
augmented_df

isg_osm_id source  highway_id  volume   original_ids single_filled  \
0     20128960001   RMBA           1     693    20128960001          None   
1     20252811007   RMBA           7     472    20252811007          None   
2     23474123002   RMBA           1    1672    23474123002          None   
3     25832750003   RMBA           1    3167    25832750003          None   
4     29544589004   RMBA           1    4039    29544589004          None   
..            ...    ...         ...     ...            ...           ...   
83  1031548416002   RMBA           5    1318  1031548416002          None   
84  1042710160001   RMBA           1    6135  1042710160001          None   
85  1042919290001   RMBA           1    3869  1042919290001          None   
86  1049590055001   RMBA           1    3021  1049590055001          None   
87  1051445076001   RMBA           1    4536  1051445076001          None   

    iteration  
0           2  
1           2  
2           2  
3           2  
4           2  
..        ...  
83          2  
84          2  
85          2  
86          2  
87          2  

[88 rows x 7 columns]

In [6]:
print(len(augmented_df['isg_osm_id'].unique()))
print(len(augmented_df['isg_osm_id']))
print(len(Missing_IDS))
print(len(error_dict.keys()))

88
88
0
0


In [7]:
# Insert Data Into DB

database = 'isgpostgres'
user = 'postgres'
password = '123@NjMpTs'
host = '20.62.75.39'
port = '9030'

encoded_password = urllib.parse.quote(password)

schema_name='tm_new_data'
table_name=output_table

connection_string="postgresql://" + user + ":" + encoded_password + "@" + host + ":" + port + "/" + database
engine=create_engine(connection_string)

augmented_df.to_sql(table_name,engine,if_exists='append',schema=schema_name,chunksize=10000,index=False,method='multi')

88